In [2]:
## Dependencies

import pandas as pd
import pathlib as path

import requests
import json
from pprint import pprint

import numpy as np

from io import StringIO

# to read the ZIP file
import requests
import zipfile
import io



# (1) create DF for countries

#### information will be used to merge or generate vizualisations
source : https://api.worldbank.org/v2/country?format=json

In [3]:
## retrieve all the countries from world bank API

### generate DF Countries

page = 1   ## there are 6 pages in the json file
data = []
url_countries = 'https://api.worldbank.org/v2/country?format=json'

for page in range(1,7):
    response = requests.get(f"{url_countries}&page={page}")
    json_data = response.json()
    
    # Add the data from the current page
    data.extend(json_data[1])  
    
## create DF and select columns

countries_wb_df = pd.DataFrame(data)
countries_wb_df['region'] = countries_wb_df['region'].apply(lambda x: x['value'])
countries_wb_df = countries_wb_df[['id', 'iso2Code', 'name', 'region','capitalCity', 'longitude', 'latitude']]
countries_wb_df = countries_wb_df.rename(columns={
                                                    'name':'country',
                                                    'id':'iso3Code'
                                                    })


regions = (countries_wb_df['region'] != "Aggregates")
countries_wb_df = countries_wb_df[regions]


countries_wb_df.head()

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.5167
2,AFG,AF,Afghanistan,South Asia,Kabul,69.1761,34.5228
5,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.242,-8.81155
6,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.3317
7,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.5075


### import continent assignement

In [4]:
path = '../Resources_external/Countries_continents.csv'
continents = pd.read_csv(path)

In [7]:
countries_df = pd.merge(countries_wb_df, continents, on='country')

countries_df = countries_df[['country', 'iso3Code_x', 'iso2Code',  'region', 'continent', 'capitalCity',
       'longitude', 'latitude']]

countries_df = countries_df.rename(columns={'iso3Code_x' : 'iso3Code'})
    

countries_df

,country,iso3Code,iso2Code,region,continent,capitalCity,longitude,latitude
0,Aruba,ABW,AW,Latin America & Caribbean,North America,Oranjestad,-70.0167,12.5167
1,Afghanistan,AFG,AF,South Asia,Asia,Kabul,69.1761,34.5228
2,Angola,AGO,AO,Sub-Saharan Africa,Africa,Luanda,13.242,-8.81155
3,Albania,ALB,AL,Europe & Central Asia,Europe,Tirane,19.8172,41.3317
4,Andorra,AND,AD,Europe & Central Asia,Europe,Andorra la Vella,1.5218,42.5075
...,...,...,...,...,...,...,...,...
212,Kosovo,XKX,XK,Europe & Central Asia,Europe,Pristina,20.926,42.565
213,"Yemen, Rep.",YEM,YE,Middle East & North Africa,Asia,Sana'a,44.2075,15.352
214,South Africa,ZAF,ZA,Sub-Saharan Africa,Africa,Pretoria,28.1871,-25.746
215,Zambia,ZMB,ZM,Sub-Saharan Africa,Africa,Lusaka,28.2937,-15.3982


In [8]:
### Export to csv file in output

countries_df.to_csv('../Resources_Output/countries_UN_referential.csv', index=False)
